<a href="https://colab.research.google.com/github/agbamate1/Batch-project/blob/main/Exercice1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## introduction-to-machine-learning

## Exercise: Using a Trained Model on New Data

**1-Create a basic model.**

**2-Save it to disk.**

**3-Load it from disk.**

**4-Use it to make predictions about a dog who was not in the training dataset.**

In [4]:
# encore une methode de charger les données avec VSCode
import pandas as pd
import requests as rq
url_graphing="https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/graphing.py"
response=rq.get(url_graphing)

with open("graphing.py","w") as file:
  file.write(response.text)

url_dataset="https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/doggy-boot-harness.csv"
response=rq.get(url_dataset)

with open ("doggy-boot-harness.csv","wb") as file: # telecharger en mode binaire
  file.write(response.content)

df=pd.read_csv("doggy-boot-harness.csv")

print(df.head())

   boot_size  harness_size     sex  age_years
0         39            58    male       12.0
1         38            58    male        9.6
2         37            52  female        8.6
3         39            58    male       10.2
4         38            57    male        7.8


In [2]:
# Deuxieme maniere simple de charger les données en colab
import pandas as pd
!pip install statsmodels
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/graphing.py
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/doggy-boot-harness.csv

--2025-01-02 13:37:03--  https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/graphing.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21511 (21K) [text/plain]
Saving to: ‘graphing.py.1’

graphing.py.1       100%[===================>]  21.01K  --.-KB/s    in 0.001s  

2025-01-02 13:37:03 (24.6 MB/s) - ‘graphing.py.1’ saved [21511/21511]

--2025-01-02 13:37:03--  https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/doggy-boot-harness.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request

In [5]:
df=pd.read_csv("doggy-boot-harness.csv")
print(df.head())

   boot_size  harness_size     sex  age_years
0         39            58    male       12.0
1         38            58    male        9.6
2         37            52  female        8.6
3         39            58    male       10.2
4         38            57    male        7.8


**Create and train a model**

In [6]:
import statsmodels.formula.api as smf

formula="boot_size~harness_size"

regression=smf.ols(formula=formula, data=df)
model=regression.fit()

print("Model est entrainé")

print(model.summary())

Model est entrainé
                            OLS Regression Results                            
Dep. Variable:              boot_size   R-squared:                       0.569
Model:                            OLS   Adj. R-squared:                  0.560
Method:                 Least Squares   F-statistic:                     63.37
Date:                Thu, 02 Jan 2025   Prob (F-statistic):           2.53e-10
Time:                        15:34:46   Log-Likelihood:                -93.054
No. Observations:                  50   AIC:                             190.1
Df Residuals:                      48   BIC:                             193.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        5.7191      

**Save and load a model**

In [7]:
# votre model est pret mais n'est pas encore sauvegardé sur votre disck
import joblib
model_filename="./mode_doggy_boot_harness.pkl"
joblib.dump(model,model_filename)

print("Model est sauvegardé!")


Model est sauvegardé!


**Test Télécharge ton model**

In [8]:
# Telechargeons notre Model

model_loaded=joblib.load(model_filename)
print("Nous venons de télécharger notre model avec ces parametres:")
print(model_loaded.params)

Nous venons de télécharger notre model avec ces parametres:
Intercept       5.719110
harness_size    0.585925
dtype: float64


**Put it together**

On our website, we want to take the harness of our customer's dog, then calculate their dog's boot size using the model that we've already trained.
Let's put everything here together to make a function that loads the model from disk, then uses it to predict our customer's dog's boot size.

In [9]:
import joblib
def load_model_predict(harness_size):
  """ En premier temps nous allons telechargé le model sur notre disck
      et connaitre les parametre de notre model entrainé et puis
      nous allons fait une prediction sur le cm de la chaussure du Chien (boot-Dog)

  """
  load_model=joblib.load(model_filename)
  print(f"Les parametres du Model sont:\n{load_model.params}")

  #etablissons la prediction
  harness_size_pred={"harness_size":[harness_size]}

  prediction=model.predict(harness_size_pred)[0]

  return prediction

test=load_model_predict(45)

print(f"La prediction de harness_size est: {round(test,0)} cm")


Les parametres du Model sont:
Intercept       5.719110
harness_size    0.585925
dtype: float64
La prediction de harness_size est: 32.0 cm


## Real World Use

In [15]:
import joblib
def check_size_boots(select_harness_size,select_boots_size):
  """
  Le but de cette fonction est de permettre aux client de faire une bonne selection
  du pointure de la chaussure de leur chien avalanche (boot_size).
  Si une mauvause selection est fait, c'est à dire si la pointure de la
  chausure selectionnée n'est pas celle que le model entrainé estime, alors un méssage doit etre
  envoié au client pour leur notifié que leur choix doit etre modifié (gros ou petit par
  rapport à l'estimation du model)
  """
  model_filename="./mode_doggy_boot_harness.pkl"
  model_load=joblib.load(model_filename)
  model_predict=model_load.predict({"harness_size":[select_harness_size]})
  boot_estimé=round(model_predict[0],0)

  print(f"Le boot estimé est:{boot_estimé}")

  # condition de comparaison avec le boot estimé

  if select_boots_size==boot_estimé:
    print("La chaussure de votre chien avalance selectionnée est conforme:")
  elif select_boots_size<boot_estimé:
    print(f"La chaussure de votre chien avalance selectionnée est trop petite à: {boot_estimé}")
  else:
    print(f"La chaussure de votre chien avalance selectionnée est trop grande à: {boot_estimé}")


check_size_boots(select_harness_size=45,select_boots_size=40)


Le boot estimé est:32.0
La chaussure de votre chien avalance selectionnée est trop grande à: 32.0


# Neuer Abschnitt